# Version information

Date:  February 08, 2024 <br>
Pyleecan version: 1.5.1

# Converter

## How to use a converter ?

This tutorial shows the way to call converter, and in a second part we explain this architecture. <br>
So, with this converter you can convert **motor IPMSM, SPMSM, SCIM and WRSM**. <br>
This converter is between Motor-CAD and Pyleecan but the architecture is create to be reuse if you want an other converter.<br>
You can select your file Motor-CAD ".mot" directly with GUI Pyleecan, the same way as loading a json file.

Most of the work to convert Pyleecan machine to Motor-CAD is already done but it would require further contribution to finish the work

Warning, the converter may do some opproximation, please read the convertion log If you want to find equivalent, approximation or error. 

Ex log : <br>
2023-12-13 10:42:50-   INFO: Pyleecan version : 1.5.2 [Pyleecan] <br>
2023-12-13 10:42:50-   INFO: Path file use to convert : /Users\LAP17\Documents/machine_MC_P/file_mot/WRSM_salient_pole.mot [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion file .mot into dict [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion dict into obj machine [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion SYNC into MachineWRSM [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Parallel_Tooth into SlotW11 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Sync_Salient_Pole into SlotW62 [Pyleecan]<br>
2023-12-13 10:42:50-  DEBUG: No duct to convert at Rotor [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: skew, not implemented [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: End of conversion, obj machine is create [Pyleecan]

## Converter architecture

### Dictionnary

The first step to simplifying the converter is creating a dictionary from the .mot file.

The .mot file is made up of different parts starting with a title in square brackets followed by a list of values.

To example:

```yaml
[Dimensions]
Stator_Lam_Dia=130
Stator_Bore=80
Airgap=1

[Design_Options]
BPM_Rotor=Surface_Radial
Top_Bar=Round
Bottom_Bar=Rectangular 
```
 
We will therefore reproduce this architecture in a dictionary, containing dictionaries as below.

```python
other_dict = {
 "[Dimensions]" : { 
"Stator_Lam_Dia" : 130,
 "Stator_Bore" : 80,
 "Airgap" : 1,
 },
 "[Design_Options]" : {
 "BPM_Rotor" : "Surface_Radial",
 "Top_Bar" : "Round",
 "Bottom_Bar" : "Rectangular",
 },
}
```

We also have unit management. When creating the dictionary equivalent to the .mot file, another unit dictionary is also created. This dictionary will allow you to have the correct conversion of units when executing the rules.


These two dictionaries will make it possible to generalize the converter to several software programs. Indeed, this use of a dictionary can be reproduced regardless of the software, the path written in the rules will simply be different.


After converting the file into a dictionary, we can begin the conversion.

### Class presentation

#### Class Convert

In this class, we have variable use to convert, and different general methods. In fact, if you want to create an other converter you can reuse these methods.

#### ConvertMC

ConvertMC inherits from Convert and add all the methods specific for Motor-CAD conversion.

![](Images/UML_convert.png)

### Rule

Each parameter to convert is define in a rule object that creates a link between one or several parameters of both software. 
The converter will first scan the machine/dict to create all the required rules to convert the machine, then execute all of them. 

To adapt the rules to different situations there are 3 types of rules, Simple, Equation and Complex.

For each of the RuleSimple, RuleComplex and RuleEquaion classes, convert_to_P ("other" to pyleecan convertion) and convert_to_other (pyleecan to "other" convertion) methods are used to convert the parameters according to the direction of conversion.



#### Rule
In this abstract class, you can find methods to select or set value in dict or in obj machine. These methods are used to handles the unit according to the unit_dict of "other" (pyleecan parameter are always stored/defined in SI even if the GUI says otherwise)

#### RuleSimple

First there are the "simple rules" that directly link a single parameter of pyleecan to a single parameter of "other". This rule also include the possibility to define a scalling factor between the two parameters (default is 1).

For example, Motor-Cad parameterizes according to diameters while Pyleecan generally uses radius. We will therefore have a factor of two for all conversions between the diameter and the radius (and conversion between [m] and [mm]).

#### RuleEquation

Then the "Equation rules" which link a single parameter of pyleecan to a single parameter of "other" but by providing an equation that can use other parameters from both software (ex: y = a+2b -3x). 

To solve the equation more easily we must respect the following rule, the Motor-CAD parameter must be “y” and the Pyleecan parameter must be “x”. 
Concerning the parameters around a, b, c… these parameters must already be defined previously. 

For example, the rotor radius (x) corresponds to a parameter in Pyleecan. While it can be found on Motor-Cad with the diameter of the stator (y) and the dimension of the air gap (a). We therefore have the following equation “y/2 – a = x”.


#### RuleComplex

Complex rules solve all the other situations with specific functions. These rules allow the calling of functions files (set in _fct_name property) present in Pyleecan, which will simplify the conversion. These files must contain other_to_P and P_to_other functions (that will be called by the corresponding methods). The objective was to limit the use of these rules as much as possible to facilitate understanding and to reduce development time.

![](Images/UML_rules.png)

### Architecture presentation

Architecture of converter follow the one of the GUI of Pyleecan:

![](Images/Selection_machine.png)

Each of these step correspond to a set of parameters and the step must be done in order. The steps are not lamination specific (stator/rotor) to enable to reuse the code.
![](Images/Machine_dimension.png)

The general process of the converter is:
- Convert other file to other dict
- Select the correct rules and the correct pyleecan objects
- Run all the rules in the correct order

There are two main types of methods: select that expend the rule list, convert that creates the correct pyleecan objects/setup the correct properties in other_dict (depending on convert direction)

The following diagram illustrates the sequence of selections/convertion. For example, if you want to convert an IPMSM motor, the converter will find the type of motor. Once it finds that it is an IPMSM, it will call the next steps, such as selecting the notch type.

All methods performing the selection begin with "select". They are all in the Convert class. The objective of these methods is to add the correct rules according to the needs and to direct to the other selection files.

The selections in these methods are made from Pyleecan machine objects (e.g. if isinstance(slot, SlotW11)), to maintain the possibility of reusing these methods.
However, to always have this selection possible, methods specific to the converter (present in the convertMC class) are necessary.

![](Images/schematics_architecture_converter.png)

During this process the rules are added in a precise order in a list called rules_list.
At the end of the selection initiated by the selection_machine_rules method we find ourselves with a list of rules and a machine with all the right classes. For example, the machine will be an IPMSM, with a slotW21 on the stator and a HoleM50 on the rotor. We have the correct definition of the elements in the machine, but the parameters such as dimensions are not defined yet.

The last step is to run all the rules present in a list to complete the machine objects properties.

Here is the simplified structure of the converter in the sense of converting a MOTOR-CAD file to a Pyleecan machine.

![](Images/Architecture_Converter.png)

### Reversibility

The converter can perform both directions of conversion. And the operation of the converter does not change depending on the direction.

Most of the work to convert Pyleecan machine to Motor-CAD is already done but it would require further contribution to finish the work

![](Images/convert_mot_to_dict.png)

Below is the organization of the converter files in the different classes:
![](Images/structure_converter.png)